In [1]:
import pandas as pd
import numpy as np

In [2]:

import requests

url = "https://covid-193.p.rapidapi.com/statistics"

headers = {
	"x-rapidapi-key": "feac4f2fd1mshc3cd47c01237b1dp1396b4jsn7fe5d960e4bc",
	"x-rapidapi-host": "covid-193.p.rapidapi.com"
}

response = requests.get(url, headers=headers)
data=(response.json())

In [3]:
stats=data["response"]

In [4]:
stats

[{'continent': 'Africa',
  'country': 'Saint-Helena',
  'population': 6115,
  'cases': {'new': None,
   'active': 2164,
   'critical': None,
   'recovered': 2,
   '1M_pop': '354211',
   'total': 2166},
  'deaths': {'new': None, '1M_pop': None, 'total': None},
  'tests': {'1M_pop': None, 'total': None},
  'day': '2025-01-16',
  'time': '2025-01-16T13:45:09+00:00'},
 {'continent': 'South-America',
  'country': 'Falkland-Islands',
  'population': 3539,
  'cases': {'new': None,
   'active': 0,
   'critical': None,
   'recovered': 1930,
   '1M_pop': '545352',
   'total': 1930},
  'deaths': {'new': None, '1M_pop': None, 'total': None},
  'tests': {'1M_pop': '2439107', 'total': 8632},
  'day': '2025-01-16',
  'time': '2025-01-16T13:45:09+00:00'},
 {'continent': 'North-America',
  'country': 'Montserrat',
  'population': 4965,
  'cases': {'new': None,
   'active': 19,
   'critical': None,
   'recovered': 1376,
   '1M_pop': '282578',
   'total': 1403},
  'deaths': {'new': None, '1M_pop': '1611'

In [5]:
# Convert JSON to DataFrame
df = pd.json_normalize(stats)

# Extract relevant columns and compute confirmed cases
df_cleaned = pd.DataFrame({
    'country': df['country'],
    'active_cases': df['cases.active'],
    'recovered_cases': df['cases.recovered'],
    'total_deaths': df['deaths.total'],
    'confirmed_cases': df['cases.active'].fillna(0) +
                       df['cases.recovered'].fillna(0) +
                       df['deaths.total'].fillna(0),
    'Date': df['day']
    
})

# Display the cleaned DataFrame
print(df_cleaned)


              country  active_cases  recovered_cases  total_deaths  \
0        Saint-Helena        2164.0              2.0           NaN   
1    Falkland-Islands           0.0           1930.0           NaN   
2          Montserrat          19.0           1376.0           8.0   
3    Diamond-Princess           0.0            699.0          13.0   
4        Vatican-City           0.0             29.0           NaN   
..                ...           ...              ...           ...   
233           S-Korea           0.0       34535939.0       35934.0   
234             Japan           NaN              NaN       74694.0   
235             Italy      165544.0       26361218.0      196487.0   
236                UK           0.0       24678275.0      232112.0   
237            Russia      175641.0       23545818.0      402756.0   

     confirmed_cases        Date  
0             2166.0  2025-01-16  
1             1930.0  2025-01-16  
2             1403.0  2025-01-16  
3              712.

In [6]:
df_cleaned.Date.value_counts()

Date
2025-01-16    238
Name: count, dtype: int64

In [7]:

# maximum date
df_cleaned['Date'].max()

'2025-01-16'

In [8]:

# importing prophet library
from prophet import Prophet
m=Prophet()

e:\COVID_19\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
# Importing plotly library
import plotly
import plotly.express as px

In [10]:
world=df_cleaned
world

,country,active_cases,recovered_cases,total_deaths,confirmed_cases,Date
0,Saint-Helena,2164.0,2.0,NaN,2166.0,2025-01-16
1,Falkland-Islands,0.0,1930.0,NaN,1930.0,2025-01-16
2,Montserrat,19.0,1376.0,8.0,1403.0,2025-01-16
3,Diamond-Princess,0.0,699.0,13.0,712.0,2025-01-16
4,Vatican-City,0.0,29.0,NaN,29.0,2025-01-16
...,...,...,...,...,...,...
233,S-Korea,0.0,34535939.0,35934.0,34571873.0,2025-01-16
234,Japan,NaN,NaN,74694.0,74694.0,2025-01-16
235,Italy,165544.0,26361218.0,196487.0,26723249.0,2025-01-16
236,UK,0.0,24678275.0,232112.0,24910387.0,2025-01-16


In [11]:
!pip install nbformat --upgrade

In [12]:
#  visualizing
figure=px.choropleth(world,locations='country',
                     locationmode='country names',
                     color='active_cases',
                     hover_name='country', range_color=[1,20000],
                     color_continuous_scale='reds',
                     title='Countries with Active cases')

figure.show

<bound method BaseFigure.show of Figure({
    'data': [{'coloraxis': 'coloraxis',
              'geo': 'geo',
              'hovertemplate': ('<b>%{hovertext}</b><br><br>cou' ... 'tive_cases=%{z}<extra></extra>'),
              'hovertext': array(['Saint-Helena', 'Falkland-Islands', 'Montserrat', ..., 'Italy', 'UK',
                                  'Russia'], dtype=object),
              'locationmode': 'country names',
              'locations': array(['Saint-Helena', 'Falkland-Islands', 'Montserrat', ..., 'Italy', 'UK',
                                  'Russia'], dtype=object),
              'name': '',
              'type': 'choropleth',
              'z': array([2.16400e+03, 0.00000e+00, 1.90000e+01, ..., 1.65544e+05, 0.00000e+00,
                          1.75641e+05])}],
    'layout': {'coloraxis': {'cmax': 20000,
                             'cmin': 1,
                             'colorbar': {'title': {'text': 'active_cases'}},
                             'colorscale': [[0.0

In [13]:
import plotly.io as pio
pio.renderers.default = "browser"
figure.show()